<a href="https://colab.research.google.com/github/heko0826/HobbyColabProjects/blob/main/spatialchat_count_people.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# スクレイピングでspatialchatに今参加している人数をチェックしよう
- 参考URL
  - https://codechacha.com/ja/selenium-explicit-implicit-wait/
  - https://trend-tracer.com/screenshot-python/
  - https://office54.net/python/scraping/selenium-click-exception#:~:text=ElementClickInterceptedException%E3%81%A8%E3%81%AF,-%E3%82%A8%E3%83%A9%E3%83%BC%EF%BC%88%E4%BE%8B%E5%A4%96%EF%BC%89%E5%86%85%E5%AE%B9&text=%E3%81%93%E3%81%AE%E3%82%A8%E3%83%A9%E3%83%BC%E3%81%AF%E3%80%8C%E3%83%A1%E3%83%83%E3%82%BB%E3%83%BC%E3%82%B8%EF%BC%9A%E3%82%A8%E3%83%AC%E3%83%A1%E3%83%B3%E3%83%88,%E3%81%AA%E3%81%8B%E3%81%A3%E3%81%9F%E3%80%8D%E3%81%A8%E3%81%84%E3%81%86%E5%86%85%E5%AE%B9%E3%81%A7%E3%81%99%E3%80%82
  - https://qiita.com/kinpira/items/383b0fbee6bf229ea03d

In [ ]:
#@title seleniumのinstall
!pip install selenium
# !pip freeze

In [ ]:
#@title chromeをヘッドレス実行するために必要なものをinstallする

!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver/ usr/bin

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
#@title importなど、実行するために必要なことを準備する
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
# 待機用ライブラリ
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [ ]:
BASE_URL = ""
password = ""
PASS_URL = BASE_URL + "?sp=" + password

In [ ]:
try:
  driver = webdriver.Chrome("chromedriver", options=options)
  driver.set_page_load_timeout(10)
  driver.set_window_size("1200", "1000")
  driver.get(PASS_URL)
  # ページ上の全ての要素が読み込まれるまで待機(15秒でタイムアウト判定)
  wait = WebDriverWait(driver, 30)
  # <button class="base-button space-entrance__accept-btn primary" data-qa="space-continue-button" data-v-156f8086="" form="user-about-form" type="submit">Continue</button>
  # がクリック可能になるまで待つ
  element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "space-entrance__accept-btn")))

  # ユーザー名入力
  userName = driver.find_element_by_name("userName")
  userName.send_keys("見回りくん")

  # 見回りくんが頑張ってボタンを押す処理
  button = driver.find_element_by_class_name("space-entrance__accept-btn")
  button.click()
  element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "space-entrance__accept-btn")))
  driver.implicitly_wait(10)
  button = driver.find_element_by_class_name("space-entrance__accept-btn")
  driver.execute_script("arguments[0].click();", button)
  driver.implicitly_wait(10)
  # 人数が書いてあるタグが現れるまで
  while not driver.find_element_by_class_name("space-title-bar-users-count__wrap"):
    if EC.element_to_be_clickable((By.CLASS_NAME, "dialog-device-error__action")):
      driver.find_element_by_class_name("dialog-device-error__action ").click()
      element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "space-entrance__accept-btn")))
      button = driver.find_element_by_class_name("space-entrance__accept-btn")
      driver.execute_script("arguments[0].click();", button)
    else:
      element = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "space-entrance__accept-btn")))
      button = driver.find_element_by_class_name("space-entrance__accept-btn")
      driver.execute_script("arguments[0].click();", button)


  # 人数を取得
  num = int(driver.find_element_by_class_name("space-title-bar-users-count__wrap").text) - 1
  print(f"{num}人おるよ" if num>0 else "誰もおらんやぞ")

  # 誰がいるか知りたかった
  # button = driver.find_element_by_class_name("space-sidebar-switch__button")
  # driver.execute_script("arguments[0].click();", button)
  # driver.implicitly_wait(10)

  # スクリーンショットの保存 
  driver.save_screenshot("screen.png")

  # htmlを表示
  # html = driver.page_source.encode("utf-8")
  # soup = BeautifulSoup(html, "html.parser")
  # print(soup.prettify())

  # driverを閉じることで、見回りくんを殺す
  driver.close()
except Exception as e:
  print(e)
  driver.close()

誰もおらんやぞ
